# Koopman RL Report: 03.26.2021

## OpenAI Gym Koopman Prediction Application: CartPole
To see how well (and quickly) the Koopman operator could predict future states, I used a Q-learning agent that learned a near-optimal policy for the CartPole environment and fit an approximate Koopman operator to it.

In [ ]:
import math
import numpy as np
import scipy as sp
import pykoopman as pk
from pydmd import OptDMD
import gym
from sklearn.preprocessing import KBinsDiscretizer
from typing import Tuple

In [ ]:
X = np.load('state-action-inputs.npy') # 20,000 entries
X = X[:int(X.shape[0]*0.0015)] # 30 points!

# Fit Koopman operator using closed-form solution to DMD
optdmd = OptDMD(svd_rank=15)
model_optdmd = pk.Koopman(regressor=optdmd)
model_optdmd.fit(X)

In [ ]:
env = gym.make('CartPole-v0')
koopEnv = gym.make('CartPole-v0')

Q_table = np.load('Q_table.npy')

n_bins = ( 6, 12 )
lower_bounds = [ env.observation_space.low[2], -math.radians(50) ]
upper_bounds = [ env.observation_space.high[2], math.radians(50) ]

def discretizer( _, __, angle, pole_velocity ) -> Tuple[int,...]:
    """Convert continuous state into a discrete state"""
    est = KBinsDiscretizer(n_bins=n_bins, encode='ordinal', strategy='uniform')
    est.fit([ lower_bounds, upper_bounds ])
    return tuple( map( int, est.transform([[ angle, pole_velocity ]])[0] ) )

def policy(state: tuple):
    """ Choosing an action on epsilon-greedy policy """
    return np.argmax(Q_table[state])

current_state = discretizer(*env.reset())
current_stateK = discretizer(*koopEnv.reset())
action = policy(current_state)
actionK = policy(current_state)

num_steps = 200
q_learner_reward = 0
koopman_reward = 0

for i in range(num_steps):
    # environment details
    observation, reward, done, _ = env.step(action)
    observationK, rewardK, doneK, _ = koopEnv.step(actionK)

    # keep track of rewards
    q_learner_reward += reward
    koopman_reward += rewardK

    # discretize state - hoping generator won't have to!
    new_state = discretizer(*observation)
    new_stateK = discretizer(*observationK)

    # get actions
    next_action = policy(new_state)
    prediction = model_optdmd.predict(np.array([*list(current_stateK), actionK]))
    prediction = np.round(np.real(prediction))
    next_actionK = int(prediction[-1])

    # update environments
    action = next_action
    actionK = next_actionK
    current_state = new_state
    current_stateK = new_stateK

print("Q rewards:", q_learner_reward)
print("K rewards:", koopman_reward)

We can see that the rewards are both 200 which means that the Koopman predictor works very well given good data, though there are plenty of papers on the subject. One thing we may want to look into is how well we can learn a controller from the Koopman operator, but since we are focused on the Generator operator, what we really want to see now is the predictive power of the Koopman Generator and how it can be used for control!

## Stochastic Koopman Generator Analysis

We simulated some paths from a standard Brownian Motion (drift coefficient 0 and diffusion coefficient 1) and then tried 3 different methods from two papers: 
+ Klus et al 2020 <https://arxiv.org/pdf/1909.10638.pdf>
+ Li and Duan 2020 <https://arxiv.org/ftp/arxiv/papers/2005/2005.03769.pdf>

### Simlulation of Brownian Data

We simulated 20 paths of standard BM each with 5000 steps in a time interval of size 5000 so that the time step was 1. We took each of these 20 paths to be a state variable in our state vector. Our state vector is thus comprised of 20 iid BMs. Formally, our state vector dynamics have the form
$$
    \text{d}\tilde X_t = b\text{d}t + \sigma\text{d}W_t
$$
where $b=0$ is a $n=20$ dimensional vector of 0s and $\sigma = I_{n\times n}$ is a $n\times n$ identity matrix.

### Fitting the BM data using Generator EDMD (gEDMD)

In [ ]:
from brownian import brownian

# The Diffusion process parameter.
sigma = 1
# Total time.
T = 10000
# Number of steps.
N = T # 10000
# Time step size
dt = T/N
# Number of realizations to generate.
m = 20
# Create an empty array to store the realizations.
X = np.empty((m, N+1))
# Initial values of x.
X[:, 0] = 50
brownian(X[:, 0], N, dt, sigma, out=X[:, 1:])
Z = np.roll(X,-1)[:, :-1]
X = X[:, :-1]

### Fitting the BM data using Generator EDMD (gEDMD) from Klus et al. 2020

From the learner's point of view, we assume that we are in the class of continuous Markov processes and thus that the generator is of the form
$$
    \mathcal{L}f = b\cdot\nabla_{\tilde x}f + \frac{1}{2}a:\nabla^2_{\tilde x}f 
    %= \sum_{i=1}^n b_i\frac{\partial f}{\partial \tilde{x}_i} + \frac{1}{2} \sum_{i=1}^n \sum_{j=1}^n a_{ij} \frac{\partial^2 f}{\partial \tilde{x}_i \partial \tilde{x}_j},
$$
where $a = \sigma \sigma^\top$, $\nabla^2_x$ denotes the Hessian, and $:$ denotes the double dot product. Applying the generator to each dictionary function \psi_k and assuming that we have access to a single ergodic sample with time step $dt= 1$, we can use the following finite difference estimator of $d\psi_k$:
$$
\widehat{\text{d}\psi_k}(\tilde{\mathbf{x}}_l) = \frac{1}{t}(\tilde{\mathbf{x}}_{l+1} - \tilde{\mathbf{x}}_l) \cdot \nabla\psi_k(\tilde{\mathbf{x}}_l) + \frac{1}{2t} \Big[(\tilde{\mathbf{x}}_{l+1} - \tilde{\mathbf{x}}_l)(\tilde{\mathbf{x}}_{l+1} - \tilde{\mathbf{x}}_l)^\top\Big] : \nabla^2 \psi_k(\tilde{\mathbf{x}}_l)
$$
Note that we are adopting Klus's notation here only for reference. The stochastic total differential $d\psi_k$ is a different object that the generator of the Koopman operator they are related in that the drift of the stochastic total differential is the same thing as the generator.

Next, we set up matrices for the dictionary and the generator applied to it: 
$$
    \Psi_{\mathbf{X}} = \begin{bmatrix} 
                        \psi_1(\tilde{\mathbf{x}}_1) & \dots & \psi_1(\tilde{\mathbf{x}}_m) \\
                        \vdots & \ddots & \vdots \\
                        \psi_k(\tilde{\mathbf{x}}_1) & \dots & \psi_k(\tilde{\mathbf{x}}_m) 
                    \end{bmatrix},
$$
$$
    \text{d}\Psi_{\mathbf{X}} = \begin{bmatrix} 
                        \text{d}\psi_1(\tilde{\mathbf{x}}_1) & \dots & \text{d}\psi_1(\tilde{\mathbf{x}}_m) \\
                        \vdots & \ddots & \vdots \\
                        \text{d}\psi_k(\tilde{\mathbf{x}}_1) & \dots & \text{d}\psi_k(\tilde{\mathbf{x}}_m) 
                    \end{bmatrix}
$$

The idea behind generator EDMD is that we assume that the genertor applied to the the dictionary functions can be ("approximately") expressed as a linear combination of the dictionary functions and find the coeficients of those linear combinations by minimizing $|| \text{d}\Psi_{\tilde{\mathbf{X}}} - M\Psi_{\tilde{\mathbf{X}}} ||_F$ which leads to the least-squares approximation $$ M = \text{d}\Psi_{\tilde{\mathbf{X}}} \Psi^{+}_{\tilde{\mathbf{X}}} = (\text{d}\Psi_{\tilde{\mathbf{X}}}\Psi_{\tilde{\mathbf{X}}}^\top)(\Psi_{\tilde{\mathbf{X}}}\Psi_{\tilde{\mathbf{X}}}^\top)^+ $$

Thus, we obtain the empirical estimate $L=M^T$ of the Koopman generator $\mathcal{L}$.

For the dictionary space, we chose monomials of up to order 2. We also tried monomials of order 1, which is not sufficient to pick up the diffusion term, but was successful at picking up the drift quicker. We hypothesize that this is because there are fewer terms in the regression over the dictionary functions.

Note: we used Klus's [d3s repo](https://github.com/sklus/d3s) to create the PsiX and dPsiX objects


In [ ]:
import observables
# from sympy import symbols
# from sympy.polys.monomials import itermonomials, monomial_count
# from sympy.polys.orderings import monomial_key

# Construct B matrix as seen in 3.1.2 of the reference paper
def constructB(d, k):
    Bt = np.zeros((d, k))
    if k == 1:
        Bt[0,0] = 1
    else:
        num = np.arange(d)
        Bt[num, num+1] = 1
    B = Bt.T
    return B

# Construct similar B matrix as above, but for second order monomials
def constructSecondOrderB(s, k):
    Bt = np.zeros((s, k))
    if k == 1:
        Bt[0,0] = 1
    else:
        row = 0
        for i in range(d+1, d+1+s):
            Bt[row,i] = 1
            row += 1
    B = Bt.T
    return B

d = X.shape[0]
m = X.shape[1]
s = int(d*(d+1)/2) # number of second order poly terms
rtoler=1e-02
atoler=1e-02
psi = observables.monomials(2)

# x_str = ""
# for i in range(d):
#     x_str += 'x_' + str(i) + ', '
# x_syms = symbols(x_str)
# M = itermonomials(x_syms, 2)
# sortedM = sorted(M, key=monomial_key('grlex', np.flip(x_syms)))
# print(sortedM)

Psi_X = psi(X)
Psi_X_T = Psi_X.T
nablaPsi = psi.diff(X)
nabla2Psi = psi.ddiff(X)
# print("nablaPsi Shape", nablaPsi.shape)
k = Psi_X.shape[0]

In [ ]:
# This computes dpsi_k(x) exactly as in the paper TODO: Put in exact reference here
# t = 1 is a placeholder time step, not really sure what it should be
def dpsi(k, l, t=1):
    difference = (X[:, l+1] - X[:, l])
    term_1 = (1/t) * (difference)
    term_2 = nablaPsi[k, :, l]
    term_3 = (1/(2*t)) * (difference.reshape(-1, 1) @ difference.reshape(1, -1))
    term_4 = nabla2Psi[k, :, :, l]
    return np.dot(term_1, term_2) + np.tensordot(term_3, term_4)
vectorized_dpsi = np.vectorize(dpsi)

# Construct \text{d}\Psi_X matrix
dPsi_X = np.empty((k, m))
for column in range(m-1):
    dPsi_X[:, column] = vectorized_dpsi(range(k), column)

# Calculate Koopman generator approximation
train = int(m * 0.8)
test = m - train
M = dPsi_X[:, :train] @ np.linalg.pinv(Psi_X[:, :train]) # \widehat{L}^\top
L = M.T # estimate of Koopman generator

Next, we outline 2 methods for how to identify the drift term $b$ from the diffusion equation using gEDMD which vary only in the last step of the procedure outlined below

1. Get eigenfunctions $\xi_i$ of $\widehat{L}$ and store them in $\Xi = \{ \xi_1, \cdots, \xi_n \}$.

2. Express eigenfunctions in terms of the dictionary $\psi(x) = \Xi^\top \psi(x)$

3. Calculate Koopman modes $V = B^\top \Xi^{-\top}$

4. We can then use the Koopman generator approximation $\widehat{L}$ directly and get drift $b$:
    $$ \mathcal{L}g(x) = b(x) \approx (LB)^\top  \psi(x) $$
    
4. Alternatively, express drift in terms of reduced order eigenexpansion (we decided to set our default cutoff to be one tenth of the total eigenfunctions)
    $$ (\mathcal{L}g) (x) = b(x) \approx \sum_{\ell = 1}^{cutoff} \lambda_\ell \psi_\ell(x) v_\ell $$
    where $v_\ell$ is the $\ell$th column vector of $V= B^\top \Xi$ where B is as above and $\Xi$ is the matrix with column vectors as eigenvectors of $\hat L$ and $\lambda$'s are their associated eigenvalues.


Note that the above method should work for any state dependent drift, including as a particular case, our constant drift vector $b(x) = b$.



In [ ]:
# Eigen decomposition
eig_vals, eig_vecs = sp.sparse.linalg.eigs(L) if sp.sparse.issparse(L) else sp.linalg.eig(L)
# Compute eigenfunction matrix
eig_funcs = (eig_vecs).T @ Psi_X

# Construct estimates of drift vector (b) and diffusion matrix (a) using two methods:
# 1. Directly from dictionary functions without dimension reduction  
# 2. Construct eigendecomposition and restrict its order

# Construct B matrix that selects first-order monomials (except 1) when multiplied by list of dictionary functions
B = constructB(d, k)
# Construct second order B matrix (selects second-order monomials)
second_orderB = constructSecondOrderB(s, k)

# Computed b function (sometimes denoted by \mu) without dimension reduction
L_times_B_transposed = (L @ B).T
def b(l):
    return L_times_B_transposed @ Psi_X[:, l] # (k,)

# Calculate Koopman modes
V_v1 = B.T @ np.linalg.inv((eig_vecs).T)

# The b_v2 function allows for heavy dimension reduction
# default is reducing by 90% (taking the first k/10 eigen-parts)
# TODO: Figure out correct place to take reals
def b_v2(l, num_dims=k//10, V=V_v1):
    res = 0
    for ell in range(k-1, k-num_dims, -1):
        res += eig_vals[ell] * eig_funcs[ell, l] * V[:, ell] #.reshape(-1, 1)
    return np.real(res)

By calculating the b's separetely and printing out their values, we could tell that they were (relatively) close to zero (numbers were around the +/- 1e-2 area) which is exactly what we were hoping to find. It is nice that the heavily reduced b_v2 performed well because it means that we can actually take advantage of the computational benefits.

The two methods to identify $\sigma(x)$ are as follows

1. Use the observable $g(x)\in \mathbb{R}^{m}$ where $m = d(d+1)/2$ as all monomials of order 2, i.e. $g(x) = [x_1^2,\, x_1x_2,\, x_2^2,\, x_1x_3,\, x_2x_3,\, x_3^2,\, \cdots]^\top$
2. Find $B \in \mathbb{R}^{k \times s}$ s.t. $B^\top \psi(x) = g(x)$
3.  
\begin{align}
    \mathcal{L}g(x) &= \nabla g(x) b(x) + \text{vecUpTri}(a)\\
    &= \nabla(B^\top \psi(x)) b(x) + \text{vecUpTri}(a)\\
    &= B^\top \nabla \psi(x) b(x) + \text{vecUpTri}(a)
\end{align}$\;$
> where vecUpTri($a$) is a vectorized (flattened) upper triangle of the a matrix (including the diagonal)         which follows the same indexing convention as $g(x)$ above.
       
4. Given an estimate for $\widehat{b}(x)$, and using $\mathcal{L}g(x) \approx (\widehat{L} B)^\top \psi(x)$ we can estimate the upper triangular entries (including the diagonal) of matrix $a$ as
$$ \implies \text{vecUpTri}(\hat a) \approx (\widehat{L}B)^\top \psi(x) - B^\top \nabla \psi(x) \widehat{b}(x) $$ 
Since $a = \sigma\sigma^\top$, it is symmetric, and having estimates of the upper triangular entries means that we have an estimate for $a$ as well. 

5. Alternatively, we can use similar eigendecomposition reduction trick as we did with the drift above. To do do, we express the generator applied to the second order monomials $g(x)$ in terms of reduced order eigenexpansion (we decided to set our default cutoff to be one tenth of the total eigenfunctions)
    $$ (\mathcal{L}g) (x) \approx \sum_{\ell = 1}^{cutoff} \lambda_\ell \psi_\ell(x) v_\ell $$
    where $v_\ell$ is the $\ell$th column vector of $V = B^\top \Xi^{-\top}$ where $B$ is as above and $\Xi$ is the matrix with column vectors as eigenvectors of $\hat L$ and $\lambda$'s are their associated eigenvalues.

    With this, we can estimate the upper triangle of matrix $a$ as: $\;$
    $$ \text{vecUpTri}(\hat a) \approx \sum_{\ell = 1}^{cutoff} \lambda_\ell \psi_\ell(x) v_\ell  - B^\top \nabla \psi(x) \widehat{b}(x) $$

In [ ]:
L_times_second_orderB_transpose = (L @ second_orderB).T

def a(l):
    return (L_times_second_orderB_transpose @ Psi_X[:, l]) - \
        (second_orderB.T @ nablaPsi[:, :, l] @ b(l))

V_v2 = second_orderB.T @ np.linalg.inv((eig_vecs).T)
def a_v2(l):
    return (b_v2(l, V=V_v2)) - \
        (second_orderB.T @ nablaPsi[:, :, l] @ b_v2(l))

# Reshape a vector as matrix and perform some tests
def covarianceMatrix(a_func, l):
    a_l = a_func(l)
    covariance = np.zeros((d, d))
    row = 0
    col = 0
    covariance[row, col] = a_l[0]
    col += 1
    n = 1
    while col < d:
        covariance[row, col] = a_l[n]
        covariance[col, row] = a_l[n]
        if row == col: 
            col += 1
            row = 0
        else:
            row += 1
        n +=1
    return covariance

Unfortunately, the results of both a and a_v2 are far from identity matrices. In fact, most of the entries were on the scale of +/- 1e1

We tried one final appoach that was adapted from Lu and Duan 2021. They exploit the Perron-Frobenius operator (PF)(solution to the Fokker-Planck equation) to analyze systems with non-Gaussian Levy noise in addition to Gaussian noise. We restricted their method to analyze our system which just has Guassian noise. We will describe the estimation procedure here, but for the theory, please see their paper.

First, we assume that the drift can be approximated as a linear combination of the dictionary functions, $b(x) \approx \psi(x)^\top C$ where $C$ is a $k\times d$ matrix. Next, we collect a sample of the finite differences between observations $S\in \mathbb{R}^{d\times m}$ where the jth column vector is
$$
S_j = (x_j - z_j)/h
$$
where $x_j$ is the jth snapshot vector and $z_j=x_{j+h}$ is the associated future value where we take time step $h=1$. To find the coeficient matrix $C$, we solve the following least squares problem
$$
min_{C} ||\Psi_X^\top C - S^\top||_F
$$
which leads to the solution
$$
\hat C = (\Psi_X \Psi_X)^{-1}(\Psi_XB^\top)
$$

In [ ]:
B = np.zeros((d, m))
m_range = np.arange(m)
B = X[:, m_range] - Z[:, m_range]
# print("B shape:", B.shape)
# print("Psi_X transpose shape:", Psi_X_T.shape)
PsiMult = sp.linalg.inv(Psi_X @ Psi_X_T) @ Psi_X
C = PsiMult @ B.T
# Each col of matric C represents the coeficients in a linear combo of the dictionary functions that makes up each component of the drift vector. So each c_{} 
# print("C shape:", C.shape)

b_v3 = C.T @ Psi_X
# for l in range(5):
#     print(b_v3[:, l])

This new version of b reached good results, but not quite as good as either b or b_v2 from earlier. The entries were on the scale of +/- 1e-1.

Finally, to calculate the diffusion matrix using Li and Duan's approach, we make the same assumption about each element of the diffusion matrix as being approximated as a linear combination of the dictionary functions 
$$
a_{ij}(x) \approx d_ij \cdot \psi(x)
$$
where $d_ij = [d_{ij,1},\, \cdots,\, d_{ij,k}]^\top$. For the diffusion terms, we need to collect samples of the (cross) quadratic variation
$$
B_{ij} = h^{-1}[(x_{i,1}-z_{i,1})(x_{j,1}-z_{j,1})]
$$
where $i$ and $j$ represent the $i$-th and $j$-th components of the snapshot vectors. The resulting least squares problem to find the coeficients $d_{ij}$ for each ($i$, $j$) pair is
$$
||\Psi_X^\top d_{ij} - B_{ij}||
$$
which results in the estimate 
$$
\hat{d}_{ij} = (\Psi_X\Psi_X^\top)^{-1}(\Psi_X B_{ij})
$$
TODO: Figure out how to vectorize/matrix-ize for faster computation!

In [ ]:
def a_v3(l):
    diffusionDictCoefs = np.empty((d, d, k))
    diffusionMat = np.empty((d, d))
    for i in range(d):
        for j in range(d):
            Bij = B[i]*B[j]
            diffusionDictCoefs[i, j] = PsiMult @ Bij
            diffusionMat[i, j] = np.dot(diffusionDictCoefs[i, j], Psi_X[:,l])
    return diffusionMat

This version of a  achieved the results we were hoping for and returned near identity matrices with numbers of about +/- 1e-1 in the entries outside the diagonal.